In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import json
from datetime import date, datetime, timedelta
import time
import os
import numpy as np
import requests
from requests.auth import HTTPBasicAuth
import pathlib
#from config import ACCESS_KEY,SECRET_KEY
# import polling
import boto3


# Generating Order
# API Key stored as an env variable
PL_API_KEY = 'fa49aea30c534483bbf74f6ec9bc311a' # Jason
PL_API_KEY = 'd1522b0573f548ea8233d09b9a917b74' # Conor
PLANET_API_KEY = PL_API_KEY #os.getenv('PL_API_KEY')
orders_v2_url = 'https://api.planet.com/compute/ops/orders/v2'

# set up requests to work with api
auth = HTTPBasicAuth(PLANET_API_KEY, '')
headers = {'content-type': 'application/json'}


item_type = "PSScene4Band"

In [6]:
# Load waypoints into a GeoDataFrame
waypoint_data = {
    'Waypoint' : ['Orangi River', 'Mara River', 'Sand River', 'Grumeti River', 'Lake Masek', 
                  'Lake Magadi', 'Lake Empakaai', 'Lake Magadi 2', 'Mbalageti River', 'Ruwana River', 'Talek River'],
    'latitude' : [-2.302313, -1.562928, -1.595733, -2.249034, -3.024818, -3.202214, -2.915433, -2.656248, -2.603015, -2.044819, -1.416096],
    'longitude' : [34.830777, 34.997068, 35.069241, 34.486842, 35.038474, 35.536431, 35.841355, 34.788239, 34.720511, 34.230374, 35.097661]
}


waypoint_df = pd.DataFrame(waypoint_data)
waypoint_gdf = gpd.GeoDataFrame(waypoint_df, geometry=gpd.points_from_xy(waypoint_df.longitude, waypoint_df.latitude))

# Applying WGS84 to the CRS
waypoint_gdf.crs = {'init' :'epsg:4326'} 

# waypoint_gdf

/Users/piper/anaconda2/envs/py36/lib/python3.6/site-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,Waypoint,latitude,longitude,geometry
0,Orangi River,-2.302313,34.830777,POINT (34.83078 -2.30231)
1,Mara River,-1.562928,34.997068,POINT (34.99707 -1.56293)
2,Sand River,-1.595733,35.069241,POINT (35.06924 -1.59573)
3,Grumeti River,-2.249034,34.486842,POINT (34.48684 -2.24903)
4,Lake Masek,-3.024818,35.038474,POINT (35.03847 -3.02482)
5,Lake Magadi,-3.202214,35.536431,POINT (35.53643 -3.20221)
6,Lake Empakaai,-2.915433,35.841355,POINT (35.84136 -2.91543)
7,Lake Magadi 2,-2.656248,34.788239,POINT (34.78824 -2.65625)
8,Mbalageti River,-2.603015,34.720511,POINT (34.72051 -2.60302)
9,Ruwana River,-2.044819,34.230374,POINT (34.23037 -2.04482)


In [16]:
# Converting geodataframe to Meters from Lat/Long
# Allows for square buffer to be applied (450m)
point_gdf_m = waypoint_gdf.to_crs(epsg=3395)
# point_gdf_m

# Applying the buffer, cap_style = 3 --> Square Buffer
buffer = point_gdf_m.buffer(450, cap_style=3)

# # Convert buffer back to WGS84 Lat/Long
buffer_wgs84 = buffer.to_crs(epsg=4326)


In [33]:
# Merging GDF and DF to get the Waypoint names
joined_buffer_wgs84 = pd.concat([waypoint_df,buffer_wgs84], axis=1)
joined_buffer_wgs84 = joined_buffer_wgs84.rename(columns = {0:'polygon'}).set_geometry('polygon')

joined_buffer_wgs84_drop = joined_buffer_wgs84.drop(['geometry'], axis=1)
joined_buffer_wgs84_json = joined_buffer_wgs84_drop.to_json()

# transforming to json for inclusion into Planet API
buffer_wgs84_json_parsed = json.loads(joined_buffer_wgs84_json)
buffer_wgs84_json_api = buffer_wgs84_json_parsed['features']#[0]['geometry']['coordinates']

today = datetime.isoformat(datetime.utcnow())+'Z'#(datetime.today())
start_date = datetime.isoformat(datetime.utcnow() - timedelta(7)) + 'Z'

In [34]:
# Getting Image ID's for each waypoint that has the analytic_sr dataset 
# Having to ping the Planet V1 API to return the image id's for our required filter
# Filter variables include: Center Coordinate, Date Range, Cloud Cover, Item Type and Asset Type


def build_order(index):
    geojson_geometry = {
    "type": "Point",
    "coordinates": [
        index['properties']['longitude'], index['properties']['latitude']
        ]
    }

    # get images that overlap with our AOI 
    geometry_filter = {
      "type": "GeometryFilter",
      "field_name": "geometry",
      "config": geojson_geometry
    }

    date_range_filter = {
      "type": "DateRangeFilter",
      "field_name": "acquired",
      "config": {
        "gte": start_date,
        "lte": today
      }
    }

    # only get images which have <10% cloud coverage
    cloud_cover_filter = {
      "type": "RangeFilter",
      "field_name": "cloud_cover",
      "config": {
        "lte": 0.1
      }
    }

    # combine our geo, date, cloud filters
    combined_filter = {
      "type": "AndFilter",
      "config": [geometry_filter, date_range_filter, cloud_cover_filter]
    }
    
    # API request object
    search_request = {
        "interval": "day",
        "item_types": [item_type],
        "asset_types" : "analytic_sr",
        "filter": combined_filter
    }
    
    search_result = \
      requests.post(
        'https://api.planet.com/data/v1/quick-search',
    #     'https://api.planet.com/data/v2',
        auth=HTTPBasicAuth(PLANET_API_KEY, ''),
        json=search_request)

    return search_result

In [79]:
id_list = []

for index in buffer_wgs84_json_api:
    waypoint = index["properties"]["Waypoint"]
    order = build_order(index)
    
    time.sleep(3)
    order = order.json()['features']
    print(len(order))
    print(order[0])
    


# appending Image ID to `joined_buffer_wgs84_drop_merge` if the analytic_sr is available
# Will only return image id's that meet this requirement.
    for i in order:
        #print(order)
        if "assets.analytic_sr:download" in order[0]["_permissions"]:
            id_list.append((waypoint,order[0]["id"]))

id_list = list(set(id_list))
# id_list

4
{'_links': {'_self': 'https://api.planet.com/data/v1/item-types/PSScene4Band/items/20200315_080658_06_1057', 'assets': 'https://api.planet.com/data/v1/item-types/PSScene4Band/items/20200315_080658_06_1057/assets/', 'thumbnail': 'https://tiles.planet.com/data/v1/item-types/PSScene4Band/items/20200315_080658_06_1057/thumb'}, '_permissions': ['assets.udm:download', 'assets.analytic:download', 'assets.analytic_xml:download', 'assets.analytic_dn:download', 'assets.analytic_dn_xml:download', 'assets.basic_analytic:download', 'assets.basic_analytic_rpc:download', 'assets.basic_analytic_dn:download', 'assets.basic_analytic_dn_rpc:download', 'assets.basic_analytic_xml:download', 'assets.basic_analytic_dn_xml:download', 'assets.basic_analytic_dn_nitf:download', 'assets.basic_analytic_dn_rpc_nitf:download', 'assets.basic_analytic_dn_xml_nitf:download', 'assets.basic_analytic_nitf:download', 'assets.basic_analytic_rpc_nitf:download', 'assets.basic_analytic_xml_nitf:download', 'assets.basic_udm:d

2
{'_links': {'_self': 'https://api.planet.com/data/v1/item-types/PSScene4Band/items/20200316_064026_0f46', 'assets': 'https://api.planet.com/data/v1/item-types/PSScene4Band/items/20200316_064026_0f46/assets/', 'thumbnail': 'https://tiles.planet.com/data/v1/item-types/PSScene4Band/items/20200316_064026_0f46/thumb'}, '_permissions': ['assets.udm:download', 'assets.analytic:download', 'assets.analytic_xml:download', 'assets.analytic_dn:download', 'assets.analytic_dn_xml:download', 'assets.basic_analytic:download', 'assets.basic_analytic_rpc:download', 'assets.basic_analytic_dn:download', 'assets.basic_analytic_dn_rpc:download', 'assets.basic_analytic_xml:download', 'assets.basic_analytic_dn_xml:download', 'assets.basic_analytic_dn_nitf:download', 'assets.basic_analytic_dn_rpc_nitf:download', 'assets.basic_analytic_dn_xml_nitf:download', 'assets.basic_analytic_nitf:download', 'assets.basic_analytic_rpc_nitf:download', 'assets.basic_analytic_xml_nitf:download', 'assets.basic_udm:download',

IndexError: list index out of range

In [41]:
# Merging image id's to the dataframe to maintain continuity

image_ids = pd.DataFrame(np.asarray(id_list))
image_ids.rename(columns = {0:'Waypoint', 1:'Image_ID'}, inplace = True) 
joined_buffer_wgs84_drop_merge = pd.merge(image_ids, joined_buffer_wgs84_drop, on='Waypoint')
# joined_buffer_wgs84_drop_merge

,Waypoint,Image_ID,latitude,longitude,polygon
0,Lake Magadi 2,20200315_080702_17_1057,-2.656248,34.788239,"POLYGON ((34.79228 -2.65218, 34.79228 -2.66031..."
1,Mbalageti River,20200315_080702_17_1057,-2.603015,34.720511,"POLYGON ((34.72455 -2.59895, 34.72455 -2.60708..."
2,Sand River,20200315_080645_74_1057,-1.595733,35.069241,"POLYGON ((35.07328 -1.59166, 35.07328 -1.59980..."
3,Talek River,20200315_080643_69_1057,-1.416096,35.097661,"POLYGON ((35.10170 -1.41203, 35.10170 -1.42016..."
4,Grumeti River,20200315_074308_1035,-2.249034,34.486842,"POLYGON ((34.49088 -2.24497, 34.49088 -2.25310..."
5,Mara River,20200315_080645_74_1057,-1.562928,34.997068,"POLYGON ((35.00111 -1.55886, 35.00111 -1.56700..."
6,Orangi River,20200315_080658_06_1057,-2.302313,34.830777,"POLYGON ((34.83482 -2.29825, 34.83482 -2.30638..."


In [45]:
# Converting list of tuple polygons to list of lists polygons
# This step is necessary to pull the Geometry from `joined_buffer_wgs84_drop_merge`
# and convert to a list of lists...appending to `joined_buffer_wgs84_drop_merge`.

def coord_lister(geom):
    coords = list(geom.exterior.coords)
    return (coords)

coordinates = joined_buffer_wgs84_drop_merge.polygon.apply(coord_lister)
res = []
for poly in coordinates:
    res_2 = list(map(list, poly)) 
    res.append(res_2)

joined_buffer_wgs84_drop_merge['poly_list'] = res
# joined_buffer_wgs84_drop_merge will be used as the basis for all remaining functions
# joined_buffer_wgs84_drop_merge

,Waypoint,Image_ID,latitude,longitude,polygon,poly_list
0,Lake Magadi 2,20200315_080702_17_1057,-2.656248,34.788239,"POLYGON ((34.79228 -2.65218, 34.79228 -2.66031...","[[34.79228141877853, -2.652182761619736], [34...."
1,Mbalageti River,20200315_080702_17_1057,-2.603015,34.720511,"POLYGON ((34.72455 -2.59895, 34.72455 -2.60708...","[[34.724553418778534, -2.598949585965823], [34..."
2,Sand River,20200315_080645_74_1057,-1.595733,35.069241,"POLYGON ((35.07328 -1.59166, 35.07328 -1.59980...","[[35.07328341877854, -1.5916649326258994], [35..."
3,Talek River,20200315_080643_69_1057,-1.416096,35.097661,"POLYGON ((35.10170 -1.41203, 35.10170 -1.42016...","[[35.10170341877854, -1.4120275932823005], [35..."
4,Grumeti River,20200315_074308_1035,-2.249034,34.486842,"POLYGON ((34.49088 -2.24497, 34.49088 -2.25310...","[[34.49088441877854, -2.2449675083763587], [34..."
5,Mara River,20200315_080645_74_1057,-1.562928,34.997068,"POLYGON ((35.00111 -1.55886, 35.00111 -1.56700...","[[35.00111041877854, -1.558859867631091], [35...."
6,Orangi River,20200315_080658_06_1057,-2.302313,34.830777,"POLYGON ((34.83482 -2.29825, 34.83482 -2.30638...","[[34.83481941877853, -2.298246660514488], [34...."


In [78]:
joined_buffer_wgs84_drop_merge['Image_ID']

0    20200315_080702_17_1057
1    20200315_080702_17_1057
2    20200315_080645_74_1057
3    20200315_080643_69_1057
4       20200315_074308_1035
5    20200315_080645_74_1057
6    20200315_080658_06_1057
Name: Image_ID, dtype: object

In [107]:
# Creating the URLs to activate the images...prevents latency during download
id0_url = []
for id0 in joined_buffer_wgs84_drop_merge['Image_ID']:
    id0_url_2 = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, id0)
    id0_url.append(id0_url_2)

# id0_url
joined_buffer_wgs84_drop_merge['id0_url'] = id0_url
joined_buffer_wgs84_drop_merge['id0_url']

0    https://api.planet.com/data/v1/item-types/PSSc...
1    https://api.planet.com/data/v1/item-types/PSSc...
2    https://api.planet.com/data/v1/item-types/PSSc...
3    https://api.planet.com/data/v1/item-types/PSSc...
4    https://api.planet.com/data/v1/item-types/PSSc...
5    https://api.planet.com/data/v1/item-types/PSSc...
6    https://api.planet.com/data/v1/item-types/PSSc...
Name: id0_url, dtype: object

In [108]:
# Returns JSON metadata for assets in this ID. 
# Learn more: planet.com/docs/reference/data-api/items-assets/#asset
result = []
for link in joined_buffer_wgs84_drop_merge['id0_url']:
#     print(link)
    result1 = \
      requests.get(
        link,
        auth=HTTPBasicAuth(PLANET_API_KEY, '')
      )
    result.append(result1)
joined_buffer_wgs84_drop_merge['activation_link'] = result
joined_buffer_wgs84_drop_merge['activation_link']

In [111]:
joined_buffer_wgs84_drop_merge

,Waypoint,Image_ID,latitude,longitude,polygon,poly_list,id0_url,activation_link
0,Lake Magadi 2,20200315_080702_17_1057,-2.656248,34.788239,"POLYGON ((34.79228 -2.65218, 34.79228 -2.66031...","[[34.79228141877853, -2.652182761619736], [34....",https://api.planet.com/data/v1/item-types/PSSc...,<Response [200]>
1,Mbalageti River,20200315_080702_17_1057,-2.603015,34.720511,"POLYGON ((34.72455 -2.59895, 34.72455 -2.60708...","[[34.724553418778534, -2.598949585965823], [34...",https://api.planet.com/data/v1/item-types/PSSc...,<Response [200]>
2,Sand River,20200315_080645_74_1057,-1.595733,35.069241,"POLYGON ((35.07328 -1.59166, 35.07328 -1.59980...","[[35.07328341877854, -1.5916649326258994], [35...",https://api.planet.com/data/v1/item-types/PSSc...,<Response [200]>
3,Talek River,20200315_080643_69_1057,-1.416096,35.097661,"POLYGON ((35.10170 -1.41203, 35.10170 -1.42016...","[[35.10170341877854, -1.4120275932823005], [35...",https://api.planet.com/data/v1/item-types/PSSc...,<Response [200]>
4,Grumeti River,20200315_074308_1035,-2.249034,34.486842,"POLYGON ((34.49088 -2.24497, 34.49088 -2.25310...","[[34.49088441877854, -2.2449675083763587], [34...",https://api.planet.com/data/v1/item-types/PSSc...,<Response [200]>
5,Mara River,20200315_080645_74_1057,-1.562928,34.997068,"POLYGON ((35.00111 -1.55886, 35.00111 -1.56700...","[[35.00111041877854, -1.558859867631091], [35....",https://api.planet.com/data/v1/item-types/PSSc...,<Response [200]>
6,Orangi River,20200315_080658_06_1057,-2.302313,34.830777,"POLYGON ((34.83482 -2.29825, 34.83482 -2.30638...","[[34.83481941877853, -2.298246660514488], [34....",https://api.planet.com/data/v1/item-types/PSSc...,<Response [200]>


In [120]:
# Parse out useful links

links = []
activation_link = []

# Getting Result Links
for r in joined_buffer_wgs84_drop_merge['activation_link']:
    links1 = r.json()[u"analytic_sr"]["_links"]
    links.append(links1)
links
joined_buffer_wgs84_drop_merge['links'] = links
joined_buffer_wgs84_drop_merge['links']

0    {'_self': 'https://api.planet.com/data/v1/asse...
1    {'_self': 'https://api.planet.com/data/v1/asse...
2    {'_self': 'https://api.planet.com/data/v1/asse...
3    {'_self': 'https://api.planet.com/data/v1/asse...
4    {'_self': 'https://api.planet.com/data/v1/asse...
5    {'_self': 'https://api.planet.com/data/v1/asse...
6    {'_self': 'https://api.planet.com/data/v1/asse...
Name: links, dtype: object

In [121]:
# Generating a list of activation links    
for l in joined_buffer_wgs84_drop_merge['links']:
    activation_link1 = l["activate"]
    activation_link.append(activation_link1)
activation_link
joined_buffer_wgs84_drop_merge['activation_link'] = activation_link
joined_buffer_wgs84_drop_merge['activation_link']

0    https://api.planet.com/data/v1/assets/eyJpIjog...
1    https://api.planet.com/data/v1/assets/eyJpIjog...
2    https://api.planet.com/data/v1/assets/eyJpIjog...
3    https://api.planet.com/data/v1/assets/eyJpIjog...
4    https://api.planet.com/data/v1/assets/eyJpIjog...
5    https://api.planet.com/data/v1/assets/eyJpIjog...
6    https://api.planet.com/data/v1/assets/eyJpIjog...
Name: activation_link, dtype: object

In [61]:
# Request activation of the 'visual' asset:
for a in joined_buffer_wgs84_drop_merge['activation_link']:
    activate_result = \
    requests.get(a,auth=HTTPBasicAuth(PLANET_API_KEY, ''))
    print(activate_result)

<Response [204]>
<Response [204]>
<Response [204]>
<Response [429]>
<Response [204]>
<Response [204]>
<Response [204]>


In [75]:
joined_buffer_wgs84_drop_merge['Image_ID']

0    20200315_080702_17_1057
1    20200315_080702_17_1057
2    20200315_080645_74_1057
3    20200315_080643_69_1057
4       20200315_074308_1035
5    20200315_080645_74_1057
6    20200315_080658_06_1057
Name: Image_ID, dtype: object

In [125]:
# Building the order lists starting with the product information

single_product = []
for image_id in joined_buffer_wgs84_drop_merge['Image_ID']:
    single_product_2 = [
        {
          'item_ids': [image_id], 
          'item_type': 'PSScene4Band',
          'product_bundle': 'analytic_sr'
        }
    ]
    single_product.append(single_product_2)
# print(single_product)
joined_buffer_wgs84_drop_merge['single_product'] = single_product
list(joined_buffer_wgs84_drop_merge['single_product'])

[[{'item_ids': ['20200315_080702_17_1057'],
   'item_type': 'PSScene4Band',
   'product_bundle': 'analytic_sr'}],
 [{'item_ids': ['20200315_080702_17_1057'],
   'item_type': 'PSScene4Band',
   'product_bundle': 'analytic_sr'}],
 [{'item_ids': ['20200315_080645_74_1057'],
   'item_type': 'PSScene4Band',
   'product_bundle': 'analytic_sr'}],
 [{'item_ids': ['20200315_080643_69_1057'],
   'item_type': 'PSScene4Band',
   'product_bundle': 'analytic_sr'}],
 [{'item_ids': ['20200315_074308_1035'],
   'item_type': 'PSScene4Band',
   'product_bundle': 'analytic_sr'}],
 [{'item_ids': ['20200315_080645_74_1057'],
   'item_type': 'PSScene4Band',
   'product_bundle': 'analytic_sr'}],
 [{'item_ids': ['20200315_080658_06_1057'],
   'item_type': 'PSScene4Band',
   'product_bundle': 'analytic_sr'}]]

In [126]:
# Setting the clipping boundaries

clip = []
for polygon in joined_buffer_wgs84_drop_merge['poly_list']:
    clip_aoi = {
        'type':'Polygon',
        'coordinates': [polygon] 
    }

# define the clip tool
    clip_2 = {
        'clip': {
            'aoi': clip_aoi
        }
    }
    clip.append(clip_2)
# clip
joined_buffer_wgs84_drop_merge['clip'] = clip
list(joined_buffer_wgs84_drop_merge['clip'])

[{'clip': {'aoi': {'type': 'Polygon',
    'coordinates': [[[34.79228141877853, -2.652182761619736],
      [34.79228141877853, -2.6603132247492183],
      [34.78419658122145, -2.6603132247492183],
      [34.78419658122145, -2.652182761619736],
      [34.79228141877853, -2.652182761619736]]]}}},
 {'clip': {'aoi': {'type': 'Polygon',
    'coordinates': [[[34.724553418778534, -2.598949585965823],
      [34.724553418778534, -2.607080400675525],
      [34.716468581221456, -2.607080400675525],
      [34.716468581221456, -2.598949585965823],
      [34.724553418778534, -2.598949585965823]]]}}},
 {'clip': {'aoi': {'type': 'Polygon',
    'coordinates': [[[35.07328341877854, -1.5916649326258994],
      [35.07328341877854, -1.599801059177252],
      [35.06519858122146, -1.599801059177252],
      [35.06519858122146, -1.5916649326258994],
      [35.07328341877854, -1.5916649326258994]]]}}},
 {'clip': {'aoi': {'type': 'Polygon',
    'coordinates': [[[35.10170341877854, -1.4120275932823005],
      [35.

In [127]:
# create an order request with the clipping tool
request_clip = []

for p, c in zip(single_product, clip):
    request_clip_2 = {
        'name': 'just clip',
        'products': p, #single_product,
        'tools': [c]
    }
    request_clip.append(request_clip_2)
# request_clip
joined_buffer_wgs84_drop_merge['request_clip'] = request_clip
list(joined_buffer_wgs84_drop_merge['request_clip'])

[{'name': 'just clip',
  'products': [{'item_ids': ['20200315_080702_17_1057'],
    'item_type': 'PSScene4Band',
    'product_bundle': 'analytic_sr'}],
  'tools': [{'clip': {'aoi': {'type': 'Polygon',
      'coordinates': [[[34.79228141877853, -2.652182761619736],
        [34.79228141877853, -2.6603132247492183],
        [34.78419658122145, -2.6603132247492183],
        [34.78419658122145, -2.652182761619736],
        [34.79228141877853, -2.652182761619736]]]}}}]},
 {'name': 'just clip',
  'products': [{'item_ids': ['20200315_080702_17_1057'],
    'item_type': 'PSScene4Band',
    'product_bundle': 'analytic_sr'}],
  'tools': [{'clip': {'aoi': {'type': 'Polygon',
      'coordinates': [[[34.724553418778534, -2.598949585965823],
        [34.724553418778534, -2.607080400675525],
        [34.716468581221456, -2.607080400675525],
        [34.716468581221456, -2.598949585965823],
        [34.724553418778534, -2.598949585965823]]]}}}]},
 {'name': 'just clip',
  'products': [{'item_ids': ['2020

In [134]:
for req_c in request_clip:
    print(req_c)

{'name': 'just clip', 'products': [{'item_ids': ['20200315_080702_17_1057'], 'item_type': 'PSScene4Band', 'product_bundle': 'analytic_sr'}], 'tools': [{'clip': {'aoi': {'type': 'Polygon', 'coordinates': [[[34.79228141877853, -2.652182761619736], [34.79228141877853, -2.6603132247492183], [34.78419658122145, -2.6603132247492183], [34.78419658122145, -2.652182761619736], [34.79228141877853, -2.652182761619736]]]}}}]}
{'name': 'just clip', 'products': [{'item_ids': ['20200315_080702_17_1057'], 'item_type': 'PSScene4Band', 'product_bundle': 'analytic_sr'}], 'tools': [{'clip': {'aoi': {'type': 'Polygon', 'coordinates': [[[34.724553418778534, -2.598949585965823], [34.724553418778534, -2.607080400675525], [34.716468581221456, -2.607080400675525], [34.716468581221456, -2.598949585965823], [34.724553418778534, -2.598949585965823]]]}}}]}
{'name': 'just clip', 'products': [{'item_ids': ['20200315_080645_74_1057'], 'item_type': 'PSScene4Band', 'product_bundle': 'analytic_sr'}], 'tools': [{'clip': {

In [135]:
# Creating the url for clipping 
def place_order(request, auth):
    response = requests.post(orders_v2_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response)
    
    if response.status_code == 429:
        time.sleep(3)
        response = requests.post(orders_v2_url, data=json.dumps(request), auth=auth, headers=headers)
    
        if not response.ok:
            raise Exception(response.content)

    order_id = response.json()['id']
    print(order_id)
    
    order_url = orders_v2_url + '/' + order_id
    
    return order_url


order_url = []
for req_c in request_clip:
    order_url2 = place_order(req_c, auth)
    order_url.append(order_url2)

order_url
joined_buffer_wgs84_drop_merge['order_url'] = order_url
list(joined_buffer_wgs84_drop_merge['order_url'])

<Response [202]>
8e7dc87a-bd4e-4621-b45d-42bc6d84ff6c
<Response [202]>
377d94a5-137f-4cd8-bf4d-8419bcc27784
<Response [202]>
011cc38e-160d-4573-8fd2-360f77ba5d55
<Response [202]>
6d184cae-daa4-4e72-85ab-0bf32edefcba
<Response [202]>
e2f5ff71-e1df-4d8c-a5d6-35f49816c922
<Response [202]>
11a3add6-bd72-4fd6-95f4-210365eea84f
<Response [202]>
c35a71b4-a191-4d02-a8c7-9269026da86f


['https://api.planet.com/compute/ops/orders/v2/8e7dc87a-bd4e-4621-b45d-42bc6d84ff6c',
 'https://api.planet.com/compute/ops/orders/v2/377d94a5-137f-4cd8-bf4d-8419bcc27784',
 'https://api.planet.com/compute/ops/orders/v2/011cc38e-160d-4573-8fd2-360f77ba5d55',
 'https://api.planet.com/compute/ops/orders/v2/6d184cae-daa4-4e72-85ab-0bf32edefcba',
 'https://api.planet.com/compute/ops/orders/v2/e2f5ff71-e1df-4d8c-a5d6-35f49816c922',
 'https://api.planet.com/compute/ops/orders/v2/11a3add6-bd72-4fd6-95f4-210365eea84f',
 'https://api.planet.com/compute/ops/orders/v2/c35a71b4-a191-4d02-a8c7-9269026da86f']

In [128]:
joined_buffer_wgs84_drop_merge

,Waypoint,Image_ID,latitude,longitude,polygon,poly_list,id0_url,activation_link,links,single_product,clip,request_clip
0,Lake Magadi 2,20200315_080702_17_1057,-2.656248,34.788239,"POLYGON ((34.79228 -2.65218, 34.79228 -2.66031...","[[34.79228141877853, -2.652182761619736], [34....",https://api.planet.com/data/v1/item-types/PSSc...,https://api.planet.com/data/v1/assets/eyJpIjog...,{'_self': 'https://api.planet.com/data/v1/asse...,"[{'item_ids': ['20200315_080702_17_1057'], 'it...","{'clip': {'aoi': {'type': 'Polygon', 'coordina...","{'name': 'just clip', 'products': [{'item_ids'..."
1,Mbalageti River,20200315_080702_17_1057,-2.603015,34.720511,"POLYGON ((34.72455 -2.59895, 34.72455 -2.60708...","[[34.724553418778534, -2.598949585965823], [34...",https://api.planet.com/data/v1/item-types/PSSc...,https://api.planet.com/data/v1/assets/eyJpIjog...,{'_self': 'https://api.planet.com/data/v1/asse...,"[{'item_ids': ['20200315_080702_17_1057'], 'it...","{'clip': {'aoi': {'type': 'Polygon', 'coordina...","{'name': 'just clip', 'products': [{'item_ids'..."
2,Sand River,20200315_080645_74_1057,-1.595733,35.069241,"POLYGON ((35.07328 -1.59166, 35.07328 -1.59980...","[[35.07328341877854, -1.5916649326258994], [35...",https://api.planet.com/data/v1/item-types/PSSc...,https://api.planet.com/data/v1/assets/eyJpIjog...,{'_self': 'https://api.planet.com/data/v1/asse...,"[{'item_ids': ['20200315_080645_74_1057'], 'it...","{'clip': {'aoi': {'type': 'Polygon', 'coordina...","{'name': 'just clip', 'products': [{'item_ids'..."
3,Talek River,20200315_080643_69_1057,-1.416096,35.097661,"POLYGON ((35.10170 -1.41203, 35.10170 -1.42016...","[[35.10170341877854, -1.4120275932823005], [35...",https://api.planet.com/data/v1/item-types/PSSc...,https://api.planet.com/data/v1/assets/eyJpIjog...,{'_self': 'https://api.planet.com/data/v1/asse...,"[{'item_ids': ['20200315_080643_69_1057'], 'it...","{'clip': {'aoi': {'type': 'Polygon', 'coordina...","{'name': 'just clip', 'products': [{'item_ids'..."
4,Grumeti River,20200315_074308_1035,-2.249034,34.486842,"POLYGON ((34.49088 -2.24497, 34.49088 -2.25310...","[[34.49088441877854, -2.2449675083763587], [34...",https://api.planet.com/data/v1/item-types/PSSc...,https://api.planet.com/data/v1/assets/eyJpIjog...,{'_self': 'https://api.planet.com/data/v1/asse...,"[{'item_ids': ['20200315_074308_1035'], 'item_...","{'clip': {'aoi': {'type': 'Polygon', 'coordina...","{'name': 'just clip', 'products': [{'item_ids'..."
5,Mara River,20200315_080645_74_1057,-1.562928,34.997068,"POLYGON ((35.00111 -1.55886, 35.00111 -1.56700...","[[35.00111041877854, -1.558859867631091], [35....",https://api.planet.com/data/v1/item-types/PSSc...,https://api.planet.com/data/v1/assets/eyJpIjog...,{'_self': 'https://api.planet.com/data/v1/asse...,"[{'item_ids': ['20200315_080645_74_1057'], 'it...","{'clip': {'aoi': {'type': 'Polygon', 'coordina...","{'name': 'just clip', 'products': [{'item_ids'..."
6,Orangi River,20200315_080658_06_1057,-2.302313,34.830777,"POLYGON ((34.83482 -2.29825, 34.83482 -2.30638...","[[34.83481941877853, -2.298246660514488], [34....",https://api.planet.com/data/v1/item-types/PSSc...,https://api.planet.com/data/v1/assets/eyJpIjog...,{'_self': 'https://api.planet.com/data/v1/asse...,"[{'item_ids': ['20200315_080658_06_1057'], 'it...","{'clip': {'aoi': {'type': 'Polygon', 'coordina...","{'name': 'just clip', 'products': [{'item_ids'..."


In [139]:
# time.sleep(600)

def poll_for_success(order_url, auth, num_loops=50, sleep_time=10):
    count = 0
    while(count < num_loops):
        r = requests.get(order_url, auth=auth)
        response = r.json()
        state = response['state']
        
        print(f'{count * sleep_time}: {state}')
        success_states = ['success', 'partial']
        
        if state == 'failed':
            raise Exception(response)
        elif state in success_states:
            break
        
        time.sleep(sleep_time)
        count += 1

In [140]:
run_clip = True

clip_order_url = []
if run_clip:
    for c in order_url:
        time.sleep(20)
        clip_order_url2 = place_order(request_clip[1], auth)
        poll_for_success(clip_order_url2,auth)
        clip_order_url.append(clip_order_url2)
        
joined_buffer_wgs84_drop_merge['clip_order_url'] = clip_order_url
list(joined_buffer_wgs84_drop_merge['clip_order_url'])

<Response [202]>
ec1cbf34-a121-4e9b-a2df-338d1d67be66
0: queued
10: queued
20: queued
30: queued
40: running
50: running
60: running
70: running
80: running
90: running
100: running
110: running
120: running
130: running
140: running
150: running
160: running
170: running
180: running
190: running
200: running
210: running
220: running
230: running
240: running
250: running
260: running
270: running
280: running
290: running
300: running
310: running
320: running
330: running
340: running
350: running
360: running
370: running
380: running
390: running
400: running
410: running
420: running
430: running
440: running
450: running
460: running
470: running
480: running
490: running
<Response [202]>
ca1b2382-74b1-41be-b9b0-9e767b16f789
0: queued
10: queued
20: queued
30: queued
40: queued
50: queued
60: running
70: running
80: running
90: running
100: running
110: running
120: running
130: running
140: running
150: running
160: running
170: running
180: running
190: running
200: running
2

['https://api.planet.com/compute/ops/orders/v2/ec1cbf34-a121-4e9b-a2df-338d1d67be66',
 'https://api.planet.com/compute/ops/orders/v2/ca1b2382-74b1-41be-b9b0-9e767b16f789',
 'https://api.planet.com/compute/ops/orders/v2/ad5c61a1-0404-43ad-baa2-7439d5cf3118',
 'https://api.planet.com/compute/ops/orders/v2/c0303578-28e9-45b7-9c16-ad3457ee726d',
 'https://api.planet.com/compute/ops/orders/v2/3407737a-5dbb-47d8-8e7e-9946d6c97149',
 'https://api.planet.com/compute/ops/orders/v2/089a01e4-e500-43be-8e47-25574e291f77',
 'https://api.planet.com/compute/ops/orders/v2/e990e4c4-bb0b-4a98-a39b-b810fc36ca72']

In [73]:
clip_order_url

['https://api.planet.com/compute/ops/orders/v2/c5f99805-1fd1-43cc-8fa0-756358e03e14',
 'https://api.planet.com/compute/ops/orders/v2/3019e928-87e7-4c48-8829-00d86ddebe8c',
 'https://api.planet.com/compute/ops/orders/v2/b617d791-550c-42cf-b002-d0e966d194cd',
 'https://api.planet.com/compute/ops/orders/v2/8d381b9e-1ca6-496e-a86a-ef52708fe90b',
 'https://api.planet.com/compute/ops/orders/v2/270310a7-4ef8-4218-9b13-d9b586fb8392',
 'https://api.planet.com/compute/ops/orders/v2/83cd1fe1-55f0-482b-8bf4-ee7c1263a781',
 'https://api.planet.com/compute/ops/orders/v2/b40acda1-e995-4147-8c28-0642709925e4']

In [138]:
# Downloading from each clip order url
def download_order(order_url, auth, overwrite=False):
    r = requests.get(order_url, auth=auth)
    print(r)

    response = r.json()
    results = response['_links']['results']
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    results_paths = [pathlib.Path(os.path.join('data', n)) for n in results_names]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name, path in zip(results_urls, results_names, results_paths):
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))
            
    return dict(zip(results_names, results_paths))


# Process sleeping to allow for activation, staging, and processing of imagery

orders_downloaded = list()
# Downloading the orders
for clip_order in clip_order_url:
    orders_downloaded.append(download_order(clip_order, auth))
    

<Response [200]>
5 items to download
data/c5f99805-1fd1-43cc-8fa0-756358e03e14/1/files/20200315_080702_17_1057_metadata.json already exists, skipping c5f99805-1fd1-43cc-8fa0-756358e03e14/1/files/20200315_080702_17_1057_metadata.json
data/c5f99805-1fd1-43cc-8fa0-756358e03e14/1/files/20200315_080702_17_1057_3B_AnalyticMS_metadata_clip.xml already exists, skipping c5f99805-1fd1-43cc-8fa0-756358e03e14/1/files/20200315_080702_17_1057_3B_AnalyticMS_metadata_clip.xml
data/c5f99805-1fd1-43cc-8fa0-756358e03e14/1/manifest.json already exists, skipping c5f99805-1fd1-43cc-8fa0-756358e03e14/1/manifest.json
data/c5f99805-1fd1-43cc-8fa0-756358e03e14/1/files/20200315_080702_17_1057_3B_AnalyticMS_SR_clip.tif already exists, skipping c5f99805-1fd1-43cc-8fa0-756358e03e14/1/files/20200315_080702_17_1057_3B_AnalyticMS_SR_clip.tif
data/c5f99805-1fd1-43cc-8fa0-756358e03e14/1/files/20200315_080702_17_1057_3B_AnalyticMS_DN_udm_clip.tif already exists, skipping c5f99805-1fd1-43cc-8fa0-756358e03e14/1/files/20200

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [80]:
joined_buffer_wgs84_drop_merge['clip_order_url'] = clip_order_url
list(joined_buffer_wgs84_drop_merge['clip_order_url'])

<Response [200]>
5 items to download
data/c5f99805-1fd1-43cc-8fa0-756358e03e14/1/files/20200315_080702_17_1057_metadata.json already exists, skipping c5f99805-1fd1-43cc-8fa0-756358e03e14/1/files/20200315_080702_17_1057_metadata.json
data/c5f99805-1fd1-43cc-8fa0-756358e03e14/1/files/20200315_080702_17_1057_3B_AnalyticMS_metadata_clip.xml already exists, skipping c5f99805-1fd1-43cc-8fa0-756358e03e14/1/files/20200315_080702_17_1057_3B_AnalyticMS_metadata_clip.xml
data/c5f99805-1fd1-43cc-8fa0-756358e03e14/1/manifest.json already exists, skipping c5f99805-1fd1-43cc-8fa0-756358e03e14/1/manifest.json
data/c5f99805-1fd1-43cc-8fa0-756358e03e14/1/files/20200315_080702_17_1057_3B_AnalyticMS_SR_clip.tif already exists, skipping c5f99805-1fd1-43cc-8fa0-756358e03e14/1/files/20200315_080702_17_1057_3B_AnalyticMS_SR_clip.tif
data/c5f99805-1fd1-43cc-8fa0-756358e03e14/1/files/20200315_080702_17_1057_3B_AnalyticMS_DN_udm_clip.tif already exists, skipping c5f99805-1fd1-43cc-8fa0-756358e03e14/1/files/20200

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [102]:
joined_buffer_wgs84_drop_merge

,Waypoint,Image_ID,latitude,longitude,polygon,poly_list
0,Lake Magadi 2,20200315_080702_17_1057,-2.656248,34.788239,"POLYGON ((34.79228 -2.65218, 34.79228 -2.66031...","[[34.79228141877853, -2.652182761619736], [34...."
1,Mbalageti River,20200315_080702_17_1057,-2.603015,34.720511,"POLYGON ((34.72455 -2.59895, 34.72455 -2.60708...","[[34.724553418778534, -2.598949585965823], [34..."
2,Sand River,20200315_080645_74_1057,-1.595733,35.069241,"POLYGON ((35.07328 -1.59166, 35.07328 -1.59980...","[[35.07328341877854, -1.5916649326258994], [35..."
3,Talek River,20200315_080643_69_1057,-1.416096,35.097661,"POLYGON ((35.10170 -1.41203, 35.10170 -1.42016...","[[35.10170341877854, -1.4120275932823005], [35..."
4,Grumeti River,20200315_074308_1035,-2.249034,34.486842,"POLYGON ((34.49088 -2.24497, 34.49088 -2.25310...","[[34.49088441877854, -2.2449675083763587], [34..."
5,Mara River,20200315_080645_74_1057,-1.562928,34.997068,"POLYGON ((35.00111 -1.55886, 35.00111 -1.56700...","[[35.00111041877854, -1.558859867631091], [35...."
6,Orangi River,20200315_080658_06_1057,-2.302313,34.830777,"POLYGON ((34.83482 -2.29825, 34.83482 -2.30638...","[[34.83481941877853, -2.298246660514488], [34...."


In [137]:
len(orders_downloaded)

4

In [103]:
file_path = list()

for order in orders_downloaded:
    for i in order:
#         print(i)

        if '_3B_AnalyticMS_SR_clip.tif' in i:
            file_path.append(i)
file_path

['c5f99805-1fd1-43cc-8fa0-756358e03e14/1/files/20200315_080702_17_1057_3B_AnalyticMS_SR_clip.tif',
 '3019e928-87e7-4c48-8829-00d86ddebe8c/1/files/20200315_080702_17_1057_3B_AnalyticMS_SR_clip.tif',
 'b617d791-550c-42cf-b002-d0e966d194cd/1/files/20200315_080702_17_1057_3B_AnalyticMS_SR_clip.tif',
 '8d381b9e-1ca6-496e-a86a-ef52708fe90b/1/files/20200315_080702_17_1057_3B_AnalyticMS_SR_clip.tif']

In [101]:
image_ids = pd.DataFrame(np.asarray(id_list))
image_ids.rename(columns = {0:'Waypoint', 1:'Image_ID'}, inplace = True) 
joined_buffer_wgs84_drop_merge = pd.merge(image_ids, joined_buffer_wgs84_drop, on='Waypoint')
# joined_buffer_wgs84_drop_merge
# 20200315_080702_17_1057

# s3_resource = boto3.resource('s3')
# s3_resource.meta.client.upload_file(
#     Filename='/Users/jasonabaker/W210/Project/Planet_API/Waypoint_Scripts_2/data/e59ba56a-c72a-4f49-b680-273e243f8fd1/1/files/20200305_163004_0f44_3B_AnalyticMS_SR_clip.tif', Bucket='w210-planet-data-api',
#     Key= '20200305_163004_0f44_3B_AnalyticMS_SR_clip.tif')

c5f99805-1fd1-43cc-8fa0-756358e03e14/1/files/20200315_080702_17_1057_metadata.json
c5f99805-1fd1-43cc-8fa0-756358e03e14/1/files/20200315_080702_17_1057_3B_AnalyticMS_metadata_clip.xml
c5f99805-1fd1-43cc-8fa0-756358e03e14/1/manifest.json
c5f99805-1fd1-43cc-8fa0-756358e03e14/1/files/20200315_080702_17_1057_3B_AnalyticMS_SR_clip.tif
c5f99805-1fd1-43cc-8fa0-756358e03e14/1/files/20200315_080702_17_1057_3B_AnalyticMS_DN_udm_clip.tif
3019e928-87e7-4c48-8829-00d86ddebe8c/1/files/20200315_080702_17_1057_metadata.json
3019e928-87e7-4c48-8829-00d86ddebe8c/1/files/20200315_080702_17_1057_3B_AnalyticMS_metadata_clip.xml
3019e928-87e7-4c48-8829-00d86ddebe8c/1/files/20200315_080702_17_1057_3B_AnalyticMS_SR_clip.tif
3019e928-87e7-4c48-8829-00d86ddebe8c/1/manifest.json
3019e928-87e7-4c48-8829-00d86ddebe8c/1/files/20200315_080702_17_1057_3B_AnalyticMS_DN_udm_clip.tif
b617d791-550c-42cf-b002-d0e966d194cd/1/files/20200315_080702_17_1057_3B_AnalyticMS_metadata_clip.xml
b617d791-550c-42cf-b002-d0e966d194cd/

['c5f99805-1fd1-43cc-8fa0-756358e03e14/1/files/20200315_080702_17_1057_3B_AnalyticMS_SR_clip.tif',
 '3019e928-87e7-4c48-8829-00d86ddebe8c/1/files/20200315_080702_17_1057_3B_AnalyticMS_SR_clip.tif',
 'b617d791-550c-42cf-b002-d0e966d194cd/1/files/20200315_080702_17_1057_3B_AnalyticMS_SR_clip.tif',
 '8d381b9e-1ca6-496e-a86a-ef52708fe90b/1/files/20200315_080702_17_1057_3B_AnalyticMS_SR_clip.tif']